In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:99%;}
div.CodeMirror {font-family:Consolas; font-size:20pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:19px;}
</style>
"""))

<font size="5" color="red">ch5 LSTM(RNN)으로 영화평 구분하기</font>
- 5만개 영화 감상평 : 타겟(종속)변수로 부정/긍정

In [2]:
# 1. 패키지 수입
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from time import time # 70.1.1부터 현재까지 몇초지났는지

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

from sklearn.metrics import confusion_matrix, f1_score

In [3]:
# 2. 하이퍼 파라미터 설정(이 파라미터를 바꾸면 정확도나 속도에 차이남)
MY_WORDS = 10000 # imdb 데이터 안의 단어 수
MY_LENGTH = 80 # 영화평 단어 수 80개만 독립변수
MY_EMBED = 32  # Embedding 결과 차원
MY_HIDDEN = 64 # LSTM의 units 차원

MY_EPOCH = 10  # 반복 학습수(fit)
MY_BATCH = 200 # 배치사이즈(fit)

In [4]:
# 3. 데이터 불러오기
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=MY_WORDS)

In [5]:
print('학습용 입력데이터(독립변수) 모양 :', X_train.shape)
print('학습용 출력데이터(종속변수) 모양 :', y_train.shape)
print('학습용 입력데이터 샘플 :',len(X_train[0]), '-', X_train[0])
print('학습용 출력데이터 샘플(0:부정/1:긍정) :', y_train[0])

print('테스트용 입력데이터(독립변수) 모양 :', X_test.shape)
print('테스트용 출력데이터(종속변수) 모양 :', y_test.shape)
print('테스트용 입력데이터 샘플 :',len(X_test[0]), '-', X_test[0])
print('테스트용 출력데이터 샘플(0:부정/1:긍정) :', y_test[0])

학습용 입력데이터(독립변수) 모양 : (25000,)
학습용 출력데이터(종속변수) 모양 : (25000,)
학습용 입력데이터 샘플 : 218 - [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16

In [6]:
# 긍정 갯수
print('학습용 데이터의 긍정 갯수 :',y_train.sum())
print('테스트용 데이터의 긍정 갯수 :',y_test.sum())

학습용 데이터의 긍정 갯수 : 12500
테스트용 데이터의 긍정 갯수 : 12500


In [7]:
# 4. 영화평 학습용 데이터 처음10개 길이 출력 함수 
def show_length():
    print('첫 10개 영화평의 길이')
    for i in range(10):
        print(f"{i+1}번째 {len(X_train[i])}")
show_length()

첫 10개 영화평의 길이
1번째 218
2번째 189
3번째 141
4번째 550
5번째 147
6번째 43
7번째 123
8번째 562
9번째 233
10번째 130


In [8]:
# 5. 문자 단어 -> 정수
word_to_id = imdb.get_word_index() # dict('문자단어':정수id)
print(word_to_id['movie'])
print(word_to_id['film'])
# 정수 -> 문자 단어
id_to_word = {} # dict (정수id:문자단어)
for key, val in word_to_id.items():
    id_to_word[val] = key
print(id_to_word[17])
print(id_to_word[19])

17
19
movie
film


In [9]:
msg = "What a wonderful movi"
msg = msg.lower().split()
# 1: 리뷰시작을 알리는 숫자, 2:문자가 짤려서 잘못 읽어옴, 3:
data = [0] + [word_to_id.get(m, -1)+3 for m in msg]
print('원 후기 내용 :', msg)
print('encoded된 data :', data)
print('data 추정 :', ' '.join([id_to_word.get(d-3, '???') for d in data]))

원 후기 내용 : ['what', 'a', 'wonderful', 'movi']
encoded된 data : [0, 51, 6, 389, 2]
data 추정 : ??? what a wonderful ???


In [10]:
# 6. 숫자 영화평 -> 자연어 영화평 함수
def decoding(review_num):
    decoded = []
    for num in review_num:
        word = id_to_word.get(num-3, '???')
        decoded.append(word)
    print(' '.join(decoded))
decoding(X_train[1]), y_train[1]

??? big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal ??? the hair is big lots of boobs ??? men wear those cut ??? shirts that show off their ??? sickening that men actually wore them and the music is just ??? trash that plays over and over again in almost every scene there is trashy music boobs and ??? taking away bodies and the gym still doesn't close for ??? all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then


(None, 0)

In [11]:
print('pad_sequence 작업전 ')
show_length()

pad_sequence 작업전 
첫 10개 영화평의 길이
1번째 218
2번째 189
3번째 141
4번째 550
5번째 147
6번째 43
7번째 123
8번째 562
9번째 233
10번째 130


In [12]:
max([len(x) for x in X_train]), min([len(x) for x in X_train])

(2494, 11)

In [13]:
# 7. 모든 영화평의 길이를 동일하게(score :75.7%)
X_train = pad_sequences(X_train,
                       padding='post',
                       truncating='post',  # 뒷 부분을 짜르고 앞부분을 남김
                       maxlen=MY_LENGTH)
X_test = pad_sequences(X_test,
                      padding='post',
                      truncating='post',
                      maxlen=MY_LENGTH)
show_length()

첫 10개 영화평의 길이
1번째 80
2번째 80
3번째 80
4번째 80
5번째 80
6번째 80
7번째 80
8번째 80
9번째 80
10번째 80


In [14]:
# 7. 모든 영화평의 길이를 동일하게
X_train = pad_sequences(X_train,
                       padding='pre',
                       truncating='pre',  # 뒷 부분을 짜르고 앞부분을 남김
                       maxlen=MY_LENGTH)
X_test = pad_sequences(X_test,
                      padding='pre',
                      truncating='pre',
                      maxlen=MY_LENGTH)
show_length()

첫 10개 영화평의 길이
1번째 80
2번째 80
3번째 80
4번째 80
5번째 80
6번째 80
7번째 80
8번째 80
9번째 80
10번째 80


In [15]:
# 8. 최종 데이터 shape 확인
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((25000, 80), (25000,), (25000, 80), (25000,))

In [19]:
# 9. 모델 생성 및 구현
model = Sequential()
model.add(Embedding(input_dim=MY_WORDS,
                    output_dim=MY_EMBED,
                    input_length=MY_LENGTH))
model.add(LSTM(units=MY_HIDDEN,
              input_shape=(MY_LENGTH, MY_EMBED),
             # kernel_initializer='he_normal', #입력 가중치 초기화
             # recurrent_initializer='orthogonal' # 순환가중치 초기화
              ))
model.add(Dense(units=1,
               activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 80, 32)            320000    
                                                                 
 lstm_2 (LSTM)               (None, 64)                24832     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 344,897
Trainable params: 344,897
Non-trainable params: 0
_________________________________________________________________


In [20]:
# 10. 학습환경 설정 및 학습하기
model.compile(loss='binary_crossentropy', # 이중분류시 손실함수
             optimizer='adam',
             metrics=['acc'])
begin = time() # 70.1.1 ~ 이 시점까지의 초수
model.fit(X_train, y_train, 
         epochs=MY_EPOCH,
         batch_size=MY_BATCH,
         verbose=1,
         validation_split=0.2)
end = time() # 70.1.1~ 이시점까지의 초수
print('총 학습 시간 : {:.2f}'.format(end-begin))

Epoch 1/10
100/100 [==============================] - 3s 14ms/step - loss: 0.5891 - acc: 0.6637 - val_loss: 0.4333 - val_acc: 0.8030
Epoch 2/10
100/100 [==============================] - 1s 9ms/step - loss: 0.3614 - acc: 0.8493 - val_loss: 0.4050 - val_acc: 0.8150
Epoch 3/10
100/100 [==============================] - 1s 9ms/step - loss: 0.2793 - acc: 0.8895 - val_loss: 0.4622 - val_acc: 0.8118
Epoch 4/10
100/100 [==============================] - 1s 9ms/step - loss: 0.2299 - acc: 0.9125 - val_loss: 0.4844 - val_acc: 0.8064
Epoch 5/10
100/100 [==============================] - 1s 9ms/step - loss: 0.1865 - acc: 0.9320 - val_loss: 0.5408 - val_acc: 0.8022
Epoch 6/10
100/100 [==============================] - 1s 9ms/step - loss: 0.1560 - acc: 0.9452 - val_loss: 0.6156 - val_acc: 0.8002
Epoch 7/10
100/100 [==============================] - 1s 9ms/step - loss: 0.1281 - acc: 0.9567 - val_loss: 0.5924 - val_acc: 0.7940
Epoch 8/10
100/100 [==============================] - 1s 9ms/step - loss: 0

In [18]:
# 11. 모델 평가
loss, acc = model.evaluate(X_test, y_test, verbose=1)
print('test 데이터 정확도(acc) :', acc)

782/782 [==============================] - 3s 3ms/step - loss: 0.8913 - acc: 0.7548
test 데이터 정확도(acc) : 0.7548400163650513


In [19]:
# 혼돈 행렬
pred = model.predict(X_test)
y_hat = (pred>0.5).astype(int).reshape(-1)
y_hat

782/782 [==============================] - 2s 2ms/step


array([0, 1, 1, ..., 0, 0, 1])

In [20]:
y_test.shape, y_hat.shape

((25000,), (25000,))

In [21]:
confusion_matrix(y_test, y_hat)

array([[9456, 3044],
       [3085, 9415]], dtype=int64)

In [22]:
# accuracy (전체 중 정답을 맞춘 비율) : 
#           = (TN+TP) / (TN+FP+FN+TP)
# precision(정밀도, 민감도 - True로 예측한 것중 맞춘 비율)
#           = TP / (FP+TP)
# recall(재현율 - 실제값이 True인 것 중 True로 맞춘 비율)
#           = TP / (FN+TP)

In [44]:
# 12. 모델 사용하기
review = """Whaa a wonderful movie The actors roles were good the content was good and the sound was good 
There was a disadvantage that I wanted to go to the bathroom in the middle because the running time was long The reservation rate was not as high as I thought so I made it easily I highly recommend it You wont regret it if you watch it
"""
import re
review = re.sub('[^a-zA-Z\s]', ' ', review)
review = review.lower().split()
review = [1]+[word_to_id.get(r, -1)+3 for r in review]
print(len(review), review)
review = np.array(review).reshape(-1, len(review))
review.shape

67 [1, 2, 6, 389, 20, 4, 156, 555, 71, 52, 4, 1500, 16, 52, 5, 4, 481, 16, 52, 50, 16, 6, 23667, 15, 13, 473, 8, 140, 8, 4, 3868, 11, 4, 655, 88, 4, 620, 58, 16, 196, 4, 14802, 967, 16, 24, 17, 312, 17, 13, 197, 38, 13, 93, 12, 714, 13, 545, 386, 12, 25, 4283, 2598, 12, 48, 25, 106, 12]


(1, 67)

In [47]:
input_data = pad_sequences(review,
                          padding='post',
                          maxlen=MY_LENGTH,
                          truncating='post' # 80개 단어 이상일 경우 짜르기
                          )
input_data.shape

(1, 80)

In [49]:
(model.predict(input_data)>0.5).astype('int8')

1/1 [==============================] - 0s 34ms/step


array([[1]], dtype=int8)

In [54]:
# 12. 모델 사용하기
review = """Whaa a wonderful movie! The actors roles were good the content was good and the sound was good 
There was a disadvantage that I wanted to go to the bathroom in the middle because the running time was long The reservation rate was not as high as I thought so I made it easily I highly recommend it You wont regret it if you watch it
"""
import re
review = re.sub('[^a-zA-Z\s]', ' ', review)
review = review.lower().split()
review

['whaa',
 'a',
 'wonderful',
 'movie',
 'the',
 'actors',
 'roles',
 'were',
 'good',
 'the',
 'content',
 'was',
 'good',
 'and',
 'the',
 'sound',
 'was',
 'good',
 'there',
 'was',
 'a',
 'disadvantage',
 'that',
 'i',
 'wanted',
 'to',
 'go',
 'to',
 'the',
 'bathroom',
 'in',
 'the',
 'middle',
 'because',
 'the',
 'running',
 'time',
 'was',
 'long',
 'the',
 'reservation',
 'rate',
 'was',
 'not',
 'as',
 'high',
 'as',
 'i',
 'thought',
 'so',
 'i',
 'made',
 'it',
 'easily',
 'i',
 'highly',
 'recommend',
 'it',
 'you',
 'wont',
 'regret',
 'it',
 'if',
 'you',
 'watch',
 'it']